In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-books-reviews/books_data.csv
/kaggle/input/amazon-books-reviews/Books_rating.csv


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Exploratory Data Analysis

In [3]:
books_rating_path = "/kaggle/input/amazon-books-reviews/Books_rating.csv"
books_data_path = "/kaggle/input/amazon-books-reviews/books_data.csv"

In [5]:
book_rating = pd.read_csv(books_rating_path, header=0)
book_rating

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,NaN,NaN,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [12]:
books_data = pd.read_csv(books_data_path, header=0)
books_data

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,"During a school trip to Ellis Island, Dominick...",['Elvira Woodruff'],http://books.google.com/books/content?id=J7M-N...,http://books.google.com/books?id=J7M-NwAACAAJ&...,Scholastic Paperbacks,2000-06-01,http://books.google.com/books?id=J7M-NwAACAAJ&...,['Juvenile Fiction'],2.0
212400,Red Boots for Christmas,Everyone in the village of Friedensdorf is hap...,NaN,http://books.google.com/books/content?id=3n8k6...,http://books.google.com/books?id=3n8k6wl4BbYC&...,NaN,1995,http://books.google.com/books?id=3n8k6wl4BbYC&...,['Juvenile Fiction'],NaN
212401,Mamaw,"Give your Mamaw a useful, beautiful and though...",['Wild Wild Cabbage'],NaN,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,2018-01-17,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,NaN
212402,The Autograph Man,Alex-Li Tandem sells autographs. His business ...,['Zadie Smith'],http://books.google.com/books/content?id=JM6YV...,http://books.google.com/books?id=JM6YVPx_clMC&...,Vintage,2003-08-12,https://play.google.com/store/books/details?id...,['Fiction'],19.0


In [26]:
np.where(books_data["Title"].isna())

(array([1066]),)

# Dataset

In [42]:
# Create a dataset, a dataloader
class TwoTowersDataset(Dataset):
    def __init__(self, df_interaction=None, df_catalog=None, 
                 interaction_csv_path=None, interaction_header=0,
                 catalog_csv_path=None, catalog_header=0, 
                 random_negative_samples=0
    ):
        super().__init__()
        self.random_negative_samples = random_negative_samples
        
        # Load interaction
        if df_interaction is not None:
            self.df_interaction = df_interaction
        elif interaction_csv_path is not None:
            self.df_interaction = pd.read_csv(interaction_csv_path, header=header)
        
        # Load catalog
        if df_catalog is not None:
            self.df_catalog = df_catalog
        elif catalog_csv_path is not None:
            self.df_catalog = pd.read_csv(catalog_csv_path, header=header)

        # Preprocessing
        self.df_catalog["bookId"] = self.df_catalog.infoLink.str.extract(r"id=([^&]+)")
        self.df_catalog.Title = self.df_catalog.Title.fillna("")
        
        # Fillna
        self.df_interaction.fillna({"User_id": "00000000000000"}, inplace=True) # Use '00000000000000' for unknown User_id
        self.df_interaction.fillna({"Id": "0000000000"}, inplace=True) # Use '0000000000' for unknown book ID
        
        # Item ID lookup
        raw_book_ids = sorted(list(set(self.df_catalog["Title"]))) # sort raw ID for reproduceability
        self.books = [raw_book_id for idx, raw_book_id in enumerate(raw_book_ids)]
        self.book2idx = {raw_book_id:idx for idx, raw_book_id in enumerate(raw_book_ids)}
        self.idx2book = {idx:raw_book_id for idx, raw_book_id in enumerate(raw_book_ids)}
        
        # User ID lookup
        raw_user_ids = sorted(list(set(self.df_interaction.User_id))) # sort raw ID for reproduceability
        self.users = [raw_user_id for raw_user_id in enumerate(raw_user_ids)]
        self.user2idx = {raw_user_id:idx for idx, raw_user_id in enumerate(raw_user_ids)}
        self.idx2user = {idx:raw_user_id for idx, raw_user_id in enumerate(raw_user_ids)}

        # Set bookId as index of catalog
        self.df_catalog.set_index("bookId", inplace=True)
        
    def __len__(self):
        return len(self.df_interaction)

    def __getitem__(self, idx):
        """Get all interactions from a user."""
        # Get positive interaction
        pos_interaction = self.df_interaction.loc[idx, :]
        user_id = pos_interaction.User_id
        pos_item_id = pos_interaction.Title
        all_pos_items = set(self.df_interaction[self.df_interaction.User_id == user_id].Title)
        # Random sample for negatives
        max_attempt = 1e3 # maximum attempt 1000 times
        all_neg_items = np.array(list(set(self.books) - all_pos_items))
        neg_sample_items = all_neg_items[np.random.randint(0, len(all_neg_items), self.random_negative_samples)]
        # Return data item
        return {
            "user": {
                "id": self.user2idx.get(user_id, 0) # 0 is unknown user ID
            },
            "items": [
                {"id": self.book2idx.get(pos_item_id, 0), "binary_score": 1},
            ] + [
                {"id": self.book2idx.get(neg_item_id, 0), "binary_score": 0} for neg_item_id in neg_sample_items
            ]
        }

In [43]:
books_rating_ds = TwoTowersDataset(
    df_interaction=book_rating, df_catalog=books_data,
    random_negative_samples = 10
)

books_rating_ds[0]

{'user': {'id': 974367},
 'items': [{'id': 83685, 'binary_score': 1},
  {'id': 135934, 'binary_score': 0},
  {'id': 104960, 'binary_score': 0},
  {'id': 135756, 'binary_score': 0},
  {'id': 118712, 'binary_score': 0},
  {'id': 95242, 'binary_score': 0},
  {'id': 149715, 'binary_score': 0},
  {'id': 17435, 'binary_score': 0},
  {'id': 45612, 'binary_score': 0},
  {'id': 104123, 'binary_score': 0},
  {'id': 128295, 'binary_score': 0}]}

# User Tower

In [44]:
# Create user tower
class UserTower(nn.Module):
    def __init__(self, num_users, id_emb_dim, tower_emb_dim):
        """
        User tower that converts user features into a user embedding for dot product.

        Args:
        num_users - total users
        id_emb_dim - Dimension of user ID embedding
        tower_emb_dim - Dimension of user tower embedding
        """
        super().__init__()
        self.id_emb = nn.Embedding(num_users, id_emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(id_emb_dim, tower_emb_dim),
            nn.ReLU(),
            nn.Linear(tower_emb_dim, tower_emb_dim)
        )

    def forward(self, user_id):
        x = self.id_emb(user_id) # [B, id_emb_dim]
        x = self.mlp(x)
        # Apply L2 normalization so to enable cosine similarity using x instead of unbounded dot product
        x = x / x.norm(dim=-1, keepdim=True)
        return x


user_id_emb_dim = 32
user_tower_emb_dim = 32
num_users = len(books_rating_ds.user2idx)

user_tower = UserTower(num_users, user_id_emb_dim, user_tower_emb_dim)
user_tower

UserTower(
  (id_emb): Embedding(1008973, 32)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
)

# Item Tower

In [11]:
# Create item tower
class ItemTower(nn.Module):
    def __init__(self, num_items, id_emb_dim, tower_emb_dim):
        super().__init__()
        self.id_emb = nn.Embedding(num_items, id_emb_dim) # [num_items, id_emb_dim]
        self.mlp = nn.Sequential(
            nn.Linear(id_emb_dim, tower_emb_dim),
            nn.ReLU(),
            nn.Linear(tower_emb_dim, tower_emb_dim)
        )

    def forward(self, item_id):
        """Calculate item tower embedding.

        Args:
        item_id - Item ID
        
        Returns:
        """
        x = self.id_emb(item_id)  # [B, id_emb_dim]
        x = self.mlp(x) # [B, tower_emb_dim]
        x = x / x.norm(dim = -1, keepdim=True) # [B, tower_emb_dim]
        return x

num_items = len(books_rating_ds.book2idx)
id_emb_dim = 32
tower_emb_dim = 32

item_tower = ItemTower(num_items, id_emb_dim, tower_emb_dim)
item_tower

ItemTower(
  (id_emb): Embedding(221998, 32)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
)

In [19]:
# Create two-towers model

class TwoTowers(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        # Item tower dimension [item_id_emb_dim, item_tower_emb_dim]
        self.item_tower = ItemTower(
            num_items = kwargs.get("num_items", 0),
            id_emb_dim = kwargs.get("item_id_emb_dim", 32),
            tower_emb_dim = kwargs.get("item_tower_emb_dim", 32),
        )
        # User tower dimension [user_id_emb_dim, user_tower_emb_dim]
        self.user_tower = UserTower(
            num_users = kwargs.get("num_users", 0),
            id_emb_dim = kwargs.get("user_id_emb_dim", 32),
            tower_emb_dim = kwargs.get("user_tower_emb_dim", 32),
        )

    def forward(self, user_id, item_id):
        """Because item_emb and user_emb are normalised in the tower. The dot product here
        is then the cosine similarity, and its value is between [-1, 1].
        """
        item_emb = self.item_tower(item_id) # [B, item_tower_dim]. Normalised into Cosine similarity 
        user_emb = self.user_tower(user_id) # [B, user_tower_dim]. Normalised into Cosine similarity
        x = (item_emb * user_emb).sum(axis=-1, keepdim=True).squeeze() # [B]. x elements are between -1 and 1.
        return x

In [ ]:
# Train / Test split

In [ ]:
# Create a in-batch negative dataset that generates negative training samples from other user's positive items
# The negative sampling dataset needs to be created for training and test individually

In [ ]:
# Train the two-towers model and monitor the loss and training epoch
# * Use Binary Cross Entropy loss
# * Switch to Negative Contrastive Ex loss

In [ ]:
# Calculate offline performance metrics on the train and test set
# Hit@K
# Recall@K
# Normalised Weighted Cumulative Gain@K
# Mean Reciprocal Rank@K